In [1]:
#抓取百度数据
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
import pandas as pd
import re
import os

from datetime import datetime,timedelta
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")
thedate2=datetime.strftime(datetime.today()-timedelta(days=1),"%m.%d")

browser = webdriver.Chrome()
url='https://motion.baidu.com/activity/yiqing_predict/main?province=%E5%85%A8%E5%9B%BD'
browser.get(url)

with open(r'./COVIDend/%s.html' % thedate,'w',encoding='utf-8') as f:
    f.write(browser.page_source)

browser.close()

In [6]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
from datetime import datetime,timedelta


city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
city_popu.head(2)


from bs4 import BeautifulSoup
import lxml
# thedate="2022-12-30"
thedate=datetime.strftime(datetime.today()-timedelta(days=1),"%Y-%m-%d")

soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

trs=soup.find_all('div',{"class":'list-item_1JNax'})

def get_result(tr):
    city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
    progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
    peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
    end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
    return [city.replace('市',''),progress,peak,end]

d=[get_result(tr) for tr in trs]

df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
df=df.sort_values(by='end')

def covid_progress(s):
    s=s.replace('%','')
    if '回落至' in s:
        s=s.replace('回落至','')
    if '上升至' in s:
        s=s.replace('上升至','')
        s=200-int(s)
    return int(s)    
df['progress_2']=df['progress'].apply(covid_progress)
print(thedate)

2023-01-04


In [20]:
# chengdu=round(69/824*100,2)
# zhengzhou=round(50/547*100,2)
# chengdu,zhengzhou

(8.37, 9.14)

In [10]:
# df=df.append({'city':'成都','progress_2':chengdu}, ignore_index=True)
# df=df.append({'city':'郑州','progress_2':zhengzhou}, ignore_index=True)
df=df.merge(city_popu,left_on='city',right_on='城市')
df['thedate']=thedate
df.to_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
print(thedate)
df.sort_values(by='progress_2')

2023-01-04


,city,progress,peak,end,progress_2,thedate,城市,GDP,人均可支配收入,常住人口
0,宿州,回落至0%,2022/12/20,2023/01/04,0,2023-01-04,宿州,2045,34373,177
1,深圳,回落至0%,2022/12/22,2023/01/04,0,2023-01-04,深圳,27670,64878,1763
2,景德镇,回落至0%,2022/12/18,2023/01/04,0,2023-01-04,景德镇,957,42283,58
3,广安,回落至0%,2022/12/17,2023/01/04,0,2023-01-04,广安,1301,38071,98
4,重庆,回落至0%,2022/12/16,2023/01/04,0,2023-01-04,重庆,25003,40006,2542
...,...,...,...,...,...,...,...,...,...,...
197,潮州,回落至23%,2022/12/25,2023/01/23,23,2023-01-04,潮州,1097,26440,175
260,北海,回落至24%,2022/12/26,2023/02/02,24,2023-01-04,北海,1277,37956,99
262,漳州,回落至29%,2022/12/27,2023/02/03,29,2023-01-04,漳州,4546,40008,93
247,临沧,回落至31%,2022/12/26,2023/01/30,31,2023-01-04,临沧,821,30794,35


In [7]:
wkbk=df.rename(columns={'peak':'搜索峰值日','end':'高峰结束日'})
title=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='citylist')
title.merge(wkbk[['city','progress','搜索峰值日','高峰结束日']],how='left',on='city')[['progress','搜索峰值日','高峰结束日']].to_clipboard(index=False)

In [8]:
#COVID END date

allpeople=df['常住人口'].sum()
# allpeople
df2=df.groupby('end').sum().reset_index()
df2['人口cumsum']=df2['常住人口'].cumsum()
df2['人口cumsum_per']=df2['人口cumsum']/allpeople
df2['sum']=np.nan
df2[['end','sum','人口cumsum_per']].to_clipboard(index=False,header=None)

In [9]:

from bs4 import BeautifulSoup
import lxml

def getpv(thedate):
    soup=BeautifulSoup(open(r'COVIDend\%s.html'  % thedate,'r',encoding='utf-8').read())

    trs=soup.find_all('div',{"class":'list-item_1JNax'})

    def get_result(tr):
        city=tr.find('div',{'class':'list-item-city_qLfui pc_UIzzf'}).find('div').text.replace('伊犁哈萨克自治州','伊犁')
        progress=tr.find('div',{'class':'rate-text_1PWuo'}).text
        peak=tr.find_all('div',{'class':'flex_3nTxm'})[1].text
        end=tr.find('div',{'class':'flex_3nTxm end_MnHhu'}).text
        return [city.replace('市',''),progress,peak,end]

    d=[get_result(tr) for tr in trs]

    df=pd.DataFrame(data=d,columns=['city','progress','peak','end'])
    df=df.sort_values(by='end')
    df=df.merge(city_popu,left_on='city',right_on='城市')

    def covid_progress(s):
        s=s.replace('%','')
        if '回落至' in s:
            s=s.replace('回落至','')
        if '上升至' in s:
            s=s.replace('上升至','')
            s=200-int(s)
        return int(s)    
    df['progress_2']=df['progress'].apply(covid_progress)

    def covid_status(s):
        status=""
        if '上升至' in s:
            s=s.replace('上升至','').replace('%','')
            if int(s) <=50:
                status='1上升至峰值的0-50%'
            elif 50< int(s) <=80:
                status='2上升至峰值的50-80%'
            elif int(s)>80:
                status='3上升至峰值的80-100%'
        if '回落至' in s:
            s=s.replace('回落至','').replace('%','')
            if int(s) <=10:
                status='8下降至峰值的0-10%'
            elif 10< int(s) <=30:
                status='7下降至峰值的10-30%'
            elif 30< int(s) <=50:
                status='6下降至峰值的30-50%'
            elif 50< int(s) <=80:
                status='5下降至峰值的50-80%'
            elif int(s)>80:
                status='4下降至至峰值的80-100%'
        return status

    df['status']=df['progress'].apply(covid_status)
    df['date']=thedate



    pv_value=pd.pivot_table(df, values='常住人口', index=['date'],columns=["status"], aggfunc=np.sum)

    pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
    return pv_value

import os
final=pd.DataFrame()
for f in os.listdir(r'COVIDend'):
    thedate=f.replace('.html','')
    final=pd.concat([final,getpv(thedate)])

final['0 sum']=1
final=final.fillna(0)
final=final.sort_index(axis=1)
final.tail(22).to_clipboard()


# pv_value.tail(3)